In [ ]:
import pandas as pd
import glob
import os

path_results = "./mpnet_base_v2_metrics_emissions/tables/"
model_name = "mpnet_base_v2"
path_model = "./mpnet_base_v2_metrics_emissions/parts/"
arquivos = glob.glob(f"{path_model}*.parquet")

df_result_metricas = pd.read_parquet(arquivos[0])
for i, e in enumerate(arquivos):
    if i == 0:
        continue

    df_result_metricas = pd.concat([df_result_metricas, pd.read_parquet(arquivos[i])], ignore_index=True)


df_result_metricas = df_result_metricas.sort_values(by=["tema", "threshold", "agg_chunk", "agg_campos", "agg_sim_tema_palavras"])

df_result_metricas = df_result_metricas.reset_index(drop=True)
df_result_metricas


In [ ]:
df_result_metricas.to_csv(f"{path_results}{model_name}_result_all_metrics_by_agg.csv", index=False)
df_result_metricas.to_parquet(f"{path_results}{model_name}_result_all_metrics_by_agg.parquet", index=False)

Criação da Tabela De Top F1's Macro por métodos de agregação

In [ ]:
import pandas as pd

chunk_agg_types = df_result_metricas["agg_chunk"].unique()
chunk_agg_types.sort()

campos_agg_types = df_result_metricas["agg_campos"].unique()
campos_agg_types.sort()

sim_tema_palavras_agg_types = df_result_metricas["agg_sim_tema_palavras"].unique()
sim_tema_palavras_agg_types.sort()

dict_top_f1 = {}

for a in chunk_agg_types:
    for b in campos_agg_types:
        # chave do índice da linha
        dict_top_f1[(a, b)] = {}
        for e in sim_tema_palavras_agg_types:
            df_filtered = df_result_metricas[
                (df_result_metricas["agg_chunk"] == a) &
                (df_result_metricas["agg_campos"] == b) &
                (df_result_metricas["agg_sim_tema_palavras"] == e)
            ]
            
            
            # Agrupa por threshold e calcula média e desvio padrão
            stats_f1_per_threshold = df_filtered.groupby("threshold")["macro_avg_f1_score"].agg(["mean", "std"])

            # Melhor threshold baseado na média, e então melhor macro e melhor desvio padrao associado
            best_threshold = stats_f1_per_threshold["mean"].idxmax()
            best_mean_f1 = stats_f1_per_threshold.loc[best_threshold, "mean"]
            best_std  = stats_f1_per_threshold.loc[best_threshold, "std"]

            # cada coluna do MultiIndex
            dict_top_f1[(a, b)][(e, "threshold")] = best_threshold 
            dict_top_f1[(a, b)][(e, "f1_macro_mean")] = best_mean_f1
            dict_top_f1[(a, b)][(e, "f1_macro_std")] = best_std

# Agora criamos o DataFrame
df_top_f1 = pd.DataFrame.from_dict(dict_top_f1, orient="index")

# Definindo MultiIndex nas linhas
df_top_f1.index = pd.MultiIndex.from_tuples(df_top_f1.index, names=["chunk_agg_tipo", "sources_agg_tipo"])

# Definindo MultiIndex nas colunas
df_top_f1.columns = pd.MultiIndex.from_tuples(df_top_f1.columns, names=["sim_tema_palavras_agg_tipo", "metric"])

df_top_f1 = df_top_f1.sort_index(axis=1)

df_top_f1

In [ ]:
df_top_f1.to_csv(f"{path_results}{model_name}_top_f1.csv")
df_top_f1.to_parquet(f"{path_results}{model_name}_top_f1.parquet")

In [ ]:
def filter_metrics(df: pd.DataFrame, metrics_to_keep=None, metrics_to_drop=None):
    """
    Filtra colunas de um DataFrame com MultiIndex de colunas,
    baseado nos valores do nível "metric".

    - metrics_to_keep: lista de métricas que você quer manter
    - metrics_to_drop: lista de métricas que você quer remover

    Retorna um novo DataFrame filtrado.
    """
    cols_metric = df.columns.get_level_values("metric")

    if metrics_to_keep is not None:
        mask = cols_metric.isin(metrics_to_keep)
    elif metrics_to_drop is not None:
        mask = ~cols_metric.isin(metrics_to_drop)
    else:
        # Se nada for passado, mantém tudo
        mask = [True] * len(df.columns)

    return df.loc[:, mask]

In [ ]:
df_top_f1_no_threshold = filter_metrics(df_top_f1, ["f1_macro_mean", "f1_macro_std"], ["threhshold"])

In [ ]:
df_top_f1_no_threshold.to_csv(f"{path_results}{model_name}_top_f1_no_threshold.csv")
df_top_f1_no_threshold.to_parquet(f"{path_results}{model_name}_top_f1_no_threshold.parquet")

In [ ]:
df_top_f1_only_f1 = filter_metrics(df_top_f1_no_threshold, ["f1_macro_mean"], ["f1_macro_std"])

In [ ]:
df_top_f1_only_f1.to_csv(f"{path_results}{model_name}_top_f1_only_f1.csv")
df_top_f1_only_f1.to_parquet(f"{path_results}{model_name}_top_f1_only_f1.parquet")

In [ ]:
df_top_f1_only_std = filter_metrics(df_top_f1_no_threshold, ["f1_macro_std"], ["f1_macro_mean"])

In [ ]:
df_top_f1_only_std.to_parquet(f"{path_results}{model_name}_top_f1_only_std.parquet")
df_top_f1_only_std.to_csv(f"{path_results}{model_name}_top_f1_only_std.csv")

Tabela F1_ALTO médio e Recall_ALTO médio associado ao threshold F1 Macro Medio máximo por cenário de agregação

In [ ]:
import pandas as pd

chunk_agg_types = df_result_metricas["agg_chunk"].unique()
chunk_agg_types.sort()

campos_agg_types = df_result_metricas["agg_campos"].unique()
campos_agg_types.sort()

sim_tema_palavras_agg_types = df_result_metricas["agg_sim_tema_palavras"].unique()
sim_tema_palavras_agg_types.sort()

dict_top_f1_ALTO_metrics = {}

for a in chunk_agg_types:
    for b in campos_agg_types:
        # chave do índice da linha
        dict_top_f1_ALTO_metrics[(a, b)] = {}
        for e in sim_tema_palavras_agg_types:
            df_filtered = df_result_metricas[
                (df_result_metricas["agg_chunk"] == a) &
                (df_result_metricas["agg_campos"] == b) &
                (df_result_metricas["agg_sim_tema_palavras"] == e)
            ]
            
            
            stats_per_threshold = df_filtered.groupby("threshold").agg({
                "macro_avg_f1_score": ["mean", "std"],
                "ALTO_f1_score": "mean",
                "ALTO_recall": "mean"   # <-- adiciona recall médio também
            })

            # Ajusta MultiIndex das colunas
            stats_per_threshold.columns = ["f1_macro_mean","f1_macro_std", "f1_ALTO_mean_assoc", "recall_ALTO_mean_assoc"]

            # Melhor threshold pelo F1 médio
            best_threshold = stats_per_threshold["f1_macro_mean"].idxmax()
            best_mean_f1   = stats_per_threshold.loc[best_threshold, "f1_macro_mean"]
            best_std       = stats_per_threshold.loc[best_threshold, "f1_macro_std"]

            #f1 ALTO e recall ALTO associados ao f1 médio máximo
            best_f1_ALTO    = stats_per_threshold.loc[best_threshold, "f1_ALTO_mean_assoc"]
            best_recall_ALTO    = stats_per_threshold.loc[best_threshold, "recall_ALTO_mean_assoc"]

            # Salvar no dict
            dict_top_f1_ALTO_metrics[(a, b)][(e, "threshold")]      = best_threshold
            dict_top_f1_ALTO_metrics[(a, b)][(e, "f1_macro_mean")]  = best_mean_f1
            dict_top_f1_ALTO_metrics[(a, b)][(e, "f1_macro_std")]   = best_std
            dict_top_f1_ALTO_metrics[(a, b)][(e, "f1_ALTO_mean_assoc")]  = best_f1_ALTO
            dict_top_f1_ALTO_metrics[(a, b)][(e, "recall_ALTO_mean_assoc")]  = best_recall_ALTO

# Agora criamos o DataFrame
df_top_f1_ALTO_metrics = pd.DataFrame.from_dict(dict_top_f1_ALTO_metrics, orient="index")

# Definindo MultiIndex nas linhas
df_top_f1_ALTO_metrics.index = pd.MultiIndex.from_tuples(df_top_f1_ALTO_metrics.index, names=["chunk_agg_tipo", "sources_agg_tipo"])

# Definindo MultiIndex nas colunas
df_top_f1_ALTO_metrics.columns = pd.MultiIndex.from_tuples(df_top_f1_ALTO_metrics.columns, names=["sim_tema_palavras_agg_tipo", "metric"])


df_top_f1_ALTO_metrics

In [ ]:
df_top_f1_ALTO_metrics.to_csv(f"{path_results}{model_name}_top_f1_ALTO_metrics.csv")
df_top_f1_ALTO_metrics.to_parquet(f"{path_results}{model_name}_top_f1_ALTO_metrics.parquet")

In [ ]:
df_top_f1_ALTO_metrics_only_f1_ALTO = filter_metrics(df_top_f1_ALTO_metrics, ["f1_ALTO_mean_assoc"], None)
df_top_f1_ALTO_metrics_only_f1_ALTO

In [ ]:
df_top_f1_ALTO_metrics_only_f1_ALTO.to_csv(f"{path_results}{model_name}_top_f1_ALTO_metrics_only_f1_ALTO.csv")
df_top_f1_ALTO_metrics_only_f1_ALTO.to_parquet(f"{path_results}{model_name}_top_f1_ALTO_metrics_only_f1_ALTO.parquet")

Tabela F1_NAO-ALTO médio e Recall_NAO-ALTO médio associado ao threshold F1 Macro Medio máximo por cenário de agregação

In [ ]:
import pandas as pd

chunk_agg_types = df_result_metricas["agg_chunk"].unique()
chunk_agg_types.sort()

campos_agg_types = df_result_metricas["agg_campos"].unique()
campos_agg_types.sort()

sim_tema_palavras_agg_types = df_result_metricas["agg_sim_tema_palavras"].unique()
sim_tema_palavras_agg_types.sort()

dict_top_f1_NAO_ALTO_metrics = {}

for a in chunk_agg_types:
    for b in campos_agg_types:
        dict_top_f1_NAO_ALTO_metrics[(a, b)] = {}
        for e in sim_tema_palavras_agg_types:
            df_filtered = df_result_metricas[
                (df_result_metricas["agg_chunk"] == a) &
                (df_result_metricas["agg_campos"] == b) &
                (df_result_metricas["agg_sim_tema_palavras"] == e)
            ]
            
            stats_per_threshold = df_filtered.groupby("threshold").agg({
                "macro_avg_f1_score": ["mean", "std"],
                "NAO-ALTO_f1_score": "mean",
                "NAO-ALTO_recall": "mean"
            })

            stats_per_threshold.columns = [
                "f1_macro_mean", "f1_macro_std",
                "f1_NAO_ALTO_mean_assoc", "recall_NAO_ALTO_mean_assoc"
            ]

            # Melhor threshold pelo F1 macro
            best_threshold = stats_per_threshold["f1_macro_mean"].idxmax()
            best_mean_f1   = stats_per_threshold.loc[best_threshold, "f1_macro_mean"]
            best_std       = stats_per_threshold.loc[best_threshold, "f1_macro_std"]

            # f1 e recall NAO-ALTO associados ao f1 macro máximo
            best_f1_NAO_ALTO    = stats_per_threshold.loc[best_threshold, "f1_NAO_ALTO_mean_assoc"]
            best_recall_NAO_ALTO = stats_per_threshold.loc[best_threshold, "recall_NAO_ALTO_mean_assoc"]

            # Salvar no dict
            dict_top_f1_NAO_ALTO_metrics[(a, b)][(e, "threshold")] = best_threshold
            dict_top_f1_NAO_ALTO_metrics[(a, b)][(e, "f1_macro_mean")] = best_mean_f1
            dict_top_f1_NAO_ALTO_metrics[(a, b)][(e, "f1_macro_std")] = best_std
            dict_top_f1_NAO_ALTO_metrics[(a, b)][(e, "f1_NAO_ALTO_mean_assoc")] = best_f1_NAO_ALTO
            dict_top_f1_NAO_ALTO_metrics[(a, b)][(e, "recall_NAO_ALTO_mean_assoc")] = best_recall_NAO_ALTO

# Criar DataFrame
df_top_f1_NAO_ALTO_metrics = pd.DataFrame.from_dict(dict_top_f1_NAO_ALTO_metrics, orient="index")

# Definir MultiIndex nas linhas
df_top_f1_NAO_ALTO_metrics.index = pd.MultiIndex.from_tuples(
    df_top_f1_NAO_ALTO_metrics.index, names=["chunk_agg_tipo", "sources_agg_tipo"]
)

# Definir MultiIndex nas colunas
df_top_f1_NAO_ALTO_metrics.columns = pd.MultiIndex.from_tuples(
    df_top_f1_NAO_ALTO_metrics.columns, names=["sim_tema_palavras_agg_tipo", "metric"]
)

In [ ]:
df_top_f1_NAO_ALTO_metrics

In [ ]:

df_top_f1_NAO_ALTO_metrics.to_csv(f"{path_results}{model_name}_top_f1_NAO_ALTO_metrics.csv")
df_top_f1_NAO_ALTO_metrics.to_parquet(f"{path_results}{model_name}_top_f1_NAO_ALTO_metrics.parquet")


In [ ]:
# Apenas f1_NAO_ALTO
df_top_f1_NAO_ALTO_metrics_only_f1_NAO_ALTO = filter_metrics(
    df_top_f1_NAO_ALTO_metrics, metrics_to_keep=["f1_NAO_ALTO_mean_assoc"]
)

In [ ]:
df_top_f1_NAO_ALTO_metrics_only_f1_NAO_ALTO.to_csv(f"{path_results}{model_name}_top_f1_NAO_ALTO_metrics_only_f1_NAO_ALTO.csv")
df_top_f1_NAO_ALTO_metrics_only_f1_NAO_ALTO.to_parquet(f"{path_results}{model_name}_top_f1_NAO_ALTO_metrics_only_f1_NAO_ALTO.parquet")

In [ ]:
import numpy as np

# 1. Pegamos só as colunas de f1_macro
df_f1 = df_top_f1.xs("f1_macro_mean", level="metric", axis=1)

# 2. Valor máximo
max_value = df_f1.to_numpy().max()

# 3. Posição (linha e coluna)
row, col = np.where(df_f1.to_numpy() == max_value)
row = row[0]
col = col[0]

# 4. Recuperar índice e coluna
row_index = df_f1.index[row]          # (chunk_agg_tipo, sources_agg_tipo)
col_index = df_f1.columns[col]        # sim_tema_palavras_agg_tipo

# 5. Recuperar o threshold correspondente
threshold_value = df_top_f1.loc[row_index, (col_index, "threshold")]

print("Maior f1_macro:", max_value)
print("Linha (chunk_agg_tipo, sources_agg_tipo):", row_index)
print("Coluna (sim_tema_palavras_agg_tipo):", col_index)
print("Threshold correspondente:", threshold_value)


In [ ]:
from matplotlib import pyplot as plt


df_filtered = df_result_metricas[
    (df_result_metricas["agg_chunk"] == row_index[0]) &
    (df_result_metricas["agg_campos"] == row_index[1]) &
    (df_result_metricas["agg_sim_tema_palavras"] == col_index)
]

histograma = df_filtered[df_filtered["threshold"] == threshold_value].macro_avg_f1_score.hist()


histograma.set
histograma.set_xlabel('f1_macro_mean')
histograma.set_ylabel('frequency')

plt.savefig(f"{path_results}{model_name}_hist__agg_chunk_{row_index[0]}__agg_campos_{row_index[1]}__agg_sim_temas_palavras_{col_index}__threshold_{int(threshold_value*100)}.png", dpi=300, bbox_inches="tight")
histograma